# Setup

On some systems, this cell needs to be run twice.

In [ ]:
%matplotlib inline
import importlib 
import numpy as np
import scipy as sp
from conservative_LMM import *
import matplotlib.pyplot as plt
from nodepy import rk
import time

# line cyclers adapted to colourblind people
from cycler import cycler
line_cycler   = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442", "#AB50B6"]) +
                 cycler(linestyle=["-", "--", "-.", ":", "-", "--", "-.", ":"]))
marker_cycler = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442", "#AB50B6"]) +
                 cycler(linestyle=["none", "none", "none", "none", "none", "none", "none", "none"]) +
                 cycler(marker=["4", "2", "3", "1", "+", "x", ".", (5, 2, 0)]))

# matplotlib's standard cycler
standard_cycler = cycler("color", ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"])

plt.rc("axes", prop_cycle=line_cycler)

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{newpxtext}\usepackage{newpxmath}\usepackage{commath}\usepackage{mathtools}")
plt.rc("font", family="serif", size=18.)
plt.rc("savefig", dpi=200)
plt.rc("legend", loc="best", fontsize="medium", fancybox=True, framealpha=0.5)
plt.rc("lines", linewidth=2.5, markersize=10, markeredgewidth=2.5)


# Energy Conservative Fourier Method for the KdV Equation

In [ ]:
N = 64
L = 80
xplot = np.linspace(0, L, N+1)
x = xplot[0:-1]
dx = x[1] - x[0]
xi = np.fft.fftfreq(N) * N * 2*np.pi / L
mu = 0.5*L
A = 2
dt = 0.1
t_final = 50000

def f(u):
    uhat = np.fft.fft(u)
    u2hat = np.fft.fft(u*u)
    
    return -(u * np.real(np.fft.ifft(1j*xi*uhat)) + np.real(np.fft.ifft(1j*xi*u2hat))) / 3. \
            - np.real(np.fft.ifft(-1j*xi*xi*xi*uhat))

def eta(w):
    return 0.5 * np.sum(w*w) * dx

def d_eta(w):
    return w * dx

def soliton(x, A):
    tmp = np.cosh(np.sqrt(3*A) / 6 * x)
    return A / (tmp * tmp)
    
def u_analytical(t, A, mu):
    c = A / 3
    x_m_ct = np.abs(np.fmod(x - c*t - x[0], L)) + x[0]
    return soliton(x_m_ct - mu, A)

u0 = u_analytical(0., A, mu)


rkm = rk.loadRKM("SDIRK23");
tt0, uu0 = relaxation_DIRK(rkm, dt, f, eta, d_eta, u0, 2, relaxed=True)

time_tmp = time.time()
tt_b, uu_b, gamma_b = conservative_BDF2(f, t_final, tt0[0], uu0[:,0], tt0[1], uu0[:,1],
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Baseline scheme:   %.2e s" % time_tmp)

time_tmp = time.time()
tt_p, uu_p, gamma_p = conservative_BDF2(f, t_final, tt0[0], uu0[:,0], tt0[1], uu0[:,1],
        return_gamma=True, projection=True, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Projection scheme: %.2e s" % time_tmp)

time_tmp = time.time()
tt_ra, uu_ra, gamma_ra = conservative_BDF2(f, t_final, tt0[0], uu0[:,0], tt0[1], uu0[:,1],
        return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Relaxation scheme: %.2e s" % time_tmp)


In [ ]:
fig_error, ax_error = plt.subplots(1, 1)
errors_b = [np.linalg.norm(uu_b[i] - u_analytical(tt_b[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt_b))]
errors_p = [np.linalg.norm(uu_p[i] - u_analytical(tt_p[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt_p))]
errors_ra = [np.linalg.norm(uu_ra[i] - u_analytical(tt_ra[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt_ra))]
plt.plot(tt_b, errors_b, label="Baseline")
plt.plot(tt_p, errors_p, label="Projection")
plt.plot(tt_ra, errors_ra, label="Relaxation")
plt.xlabel("$t$"); plt.ylabel("Error"); plt.xlim(tt_b[1], tt_b[-1])
plt.xscale("log")
plt.yscale("log")
plt.plot(tt_b, 4.e-4*tt_b**1, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(t)$", (5.e2, 6.e-2), color="gray")
plt.plot(tt_b[len(tt_b)//20:], 4.e-8*tt_b[len(tt_b)//20:]**2, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(t^2)$", (3.e3, 1.e1), color="gray")
plt.plot(tt_b[len(tt_b)//500:len(tt_b)//10], 3.e-6*tt_b[len(tt_b)//500:len(tt_b)//10]**2, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(t^2)$", (1.e2, 1.e0), color="gray")
plt.savefig("../figures/KdV_error.pdf", bbox_inches="tight")

fig_energy, ax_energy = plt.subplots(1, 1)
H_b = [eta(u) for u in uu_b]
H_p = [eta(u) for u in uu_p]
H_ra = [eta(u) for u in uu_ra]
plt.plot(tt_b, H_b - H_b[0], label="Baseline")
plt.plot(tt_p, H_p - H_p[0], label="Projection")
plt.plot(tt_ra, H_ra - H_ra[0], label="Relaxation")
plt.xlabel("$t$"); plt.ylabel("Change of Energy"); plt.xlim(tt_b[0], tt_b[-1])
plt.yscale("symlog", linthreshy=1.e-13)
plt.savefig("../figures/KdV_energy.pdf", bbox_inches="tight")

fig_solution, ax_solution = plt.subplots(1, 1)
xx = x + L/2
shift_idx = N//2
shifted_u = np.hstack((uu_b[-1][shift_idx:], uu_b[-1][:shift_idx]))
plt.plot(xx, shifted_u, label="Baseline")
shifted_u = np.hstack((uu_p[-1][shift_idx:], uu_p[-1][:shift_idx]))
plt.plot(xx, shifted_u, label="Projection")
shifted_u = np.hstack((uu_ra[-1][shift_idx:], uu_ra[-1][:shift_idx]))
plt.plot(xx, shifted_u, label="Relaxation")
shifted_u = np.hstack((u_analytical(t_final, A, mu)[shift_idx:], u_analytical(t_final, A, mu)[:shift_idx]))
plt.plot(xx, shifted_u, label="Analytical")
plt.xlabel("$x$"); plt.ylabel("$u$"); plt.xlim(xx[0], xx[-1])
plt.savefig("../figures/KdV_solution.pdf", bbox_inches="tight")

fig_mass, ax_mass = plt.subplots(1, 1)
mass_b = [sum(u)*dx for u in uu_b]
mass_p = [sum(u)*dx for u in uu_p]
mass_ra = [sum(u)*dx for u in uu_ra]
plt.plot(tt_b, mass_b, label="Baseline")
plt.plot(tt_p, mass_p, label="Projection")
plt.plot(tt_ra, mass_ra, label="Relaxation")
plt.ylabel("Mass")
plt.xlabel("$t$")
plt.xlim(tt_b[0], tt_b[-1])
plt.savefig("../figures/KdV_mass.pdf", bbox_inches="tight")

plt.figure()
handles, labels = ax_solution.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/KdV_legend.pdf", bbox_inches="tight")